In [1]:
!pip install tqdm

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm  # tqdm 라이브러리 추가

service_key = '16JJ62M2-16JJ-16JJ-16JJ-16JJ62M2IT'

num_of_rows = 1000
page = 1

filtered_data = []
has_gwangju_data = False  # 광주광역시 데이터 여부

# tqdm으로 루프를 감싸기
for _ in tqdm(range(30)):  # 예상되는 총 페이지 수 대신 큰 값(예: 10000) 사용
    url = f'http://safemap.go.kr/openApiService/data/getConvenienceStoreData.do?' \
          f'serviceKey={service_key}&numOfRows={num_of_rows}&pageNo={page}&dataType=XML'

    result = urlopen(url)
    data = result.read()  # 데이터를 바이트 형태로 읽어옴

    soup = BeautifulSoup(data, 'xml')  # 파싱할 때 'xml' 지정
    items = soup.find_all('item')

    for item in items:
        adres = item.find('ADRES').text
        if '광주광역시' in adres:
            has_gwangju_data = True
            objt_id = item.find('OBJT_ID').text
            fclty_nm = item.find('FCLTY_NM').text
            filtered_data.append({'일련번호': objt_id, '상호명': fclty_nm, '지번주소': adres})

    page += 1

    if not items:
        break  # 데이터가 없으면 종료

if not has_gwangju_data:
    print("광주광역시 데이터가 없습니다.")
else:
    for entry in filtered_data:
        print(f"일련번호: {entry['일련번호']}")
        print(f"상호명: {entry['상호명b']}")
        print(f"지번주소: {entry['지번주소']}")
        print('---')


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [01:02<00:00,  2.08s/it]

일련번호: 20397
상호명: 피씨유저SYSTEM
지번주소: 광주광역시 북구 운암동 420-16
---
일련번호: 20398
상호명: GS25
지번주소: 광주광역시 북구 용봉동 1209-1
---
일련번호: 20399
상호명: GS서영대학교점
지번주소: 광주광역시 북구 운암동 885-43
---
일련번호: 20400
상호명: 코리아세븐광주주월3호
지번주소: 광주광역시 남구 주월동 1286-1
---
일련번호: 20401
상호명: CU
지번주소: 광주광역시 서구 농성동 425-4
---
일련번호: 20402
상호명: 코리아세븐광주월산2호
지번주소: 광주광역시 남구 월산동 1048-150
---
일련번호: 20403
상호명: CU
지번주소: 광주광역시 동구 충장로5가 24
---
일련번호: 20404
상호명: GS25
지번주소: 광주광역시 광산구 우산동 1037-2
---
일련번호: 20405
상호명: GS25
지번주소: 광주광역시 광산구 월계동 829-8
---
일련번호: 20406
상호명: GS25
지번주소: 광주광역시 서구 풍암동 1058
---
일련번호: 20407
상호명: 지스탑
지번주소: 광주광역시 북구 용전동 1224-2
---
일련번호: 20408
상호명: 시유광주지석중앙점
지번주소: 광주광역시 남구 지석동 323-3
---
일련번호: 20409
상호명: 코리아세븐광주양산휴다음점
지번주소: 광주광역시 북구 양산동 249-38
---
일련번호: 20410
상호명: 세븐일레븐광주서석골드점
지번주소: 광주광역시 동구 서석동 98-1
---
일련번호: 20411
상호명: 씨유금동중앙
지번주소: 광주광역시 동구 금동 211-6
---
일련번호: 20412
상호명: 세븐일레븐광주두암라인점
지번주소: 광주광역시 북구 두암동 916-8
---
일련번호: 20413
상호명: 세븐일레븐광주신하남공단점
지번주소: 광주광역시 광산구 오선동 254-6
---
일련번호: 20414
상호명: 세븐일레븐
지번주소: 광주광역시 동구 서석동 375
---
일련번호: 20415
상호

In [12]:
import pandas as pd

# filtered_data 리스트에 데이터 추가하는 코드는 생략

if not has_gwangju_data:
    print("광주광역시 데이터가 없습니다.")
else:
    df = pd.DataFrame(filtered_data)

    excel_filename = "filtered_data_excel.xlsx"
    df.to_excel(excel_filename, index=False, encoding='utf-8')

    print(f"데이터를 {excel_filename}로 저장했습니다.")

C:\Users\gjaischool1\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


데이터를 filtered_data_excel.xlsx로 저장했습니다.


In [3]:
!pip install requests
!pip install openpyxl

In [8]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
import os
import requests
import openpyxl

In [9]:
#naver map api key
client_id = 'a8t91ib9kk';    # 본인이 할당받은 ID 입력
client_pw = 'yhe88919iBF0BX7hyGBXgiAJbatCp3Rt7e6TxGst';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 바탕화면 경로 가져오기
desktop_path = os.path.expanduser("~") + "/Desktop/"

# 주소 목록 파일 (.xlsx)
data = pd.read_excel('mart.xlsx')

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in data['지번주소']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])


np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"상호명": data['상호명'].values,
                              "도로명": data['도로명주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})

pd_geo_coordi.to_csv('martAddr.csv')

pd_geo_coordi.to_csv(os.path.join(desktop_path, 'martAddr.csv'))

print(f"데이터를 {desktop_path}martAddr.csv로 저장했습니다.")


Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S

KeyError: '도로명주소'

In [13]:
import os
import requests
import openpyxl

# Naver Map API 정보
client_id = 'a8t91ib9kk'
client_pw = 'yhe88919iBF0BX7hyGBXgiAJbatCp3Rt7e6TxGst'
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

# 바탕화면 경로 가져오기
desktop_path = os.path.expanduser("~") + "/Desktop/"

# 주소 목록 파일 (.xlsx)
data = pd.read_excel('mart.xlsx')

# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []     
for add in data['지번주소']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])

# 데이터프레임 생성 및 CSV 저장
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"상호명": data['상호명'].values,
                              "지번주소": data['지번주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})

csv_filename = os.path.join(desktop_path, 'martAddr.csv')
pd_geo_coordi.to_csv(csv_filename, index=False, encoding='CP949')

print(f"데이터를 {csv_filename}로 저장했습니다.")


Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S